In [5]:
from konlpy.tag import Okt
# kkma = Kkma()
# test = kkma.nouns('테스트는 설치 테스트 입니다')

In [6]:
from konlpy.tag import Kkma
kkma = Kkma()
test = kkma.nouns('테스트는 설치 테스트 입니다')


In [7]:
okt = Okt()

In [8]:
with open('stopwords_kor.txt',mode= 'r',encoding= 'UTF-8') as f:
    stop = f.readlines()


In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup()
stopwords = []
for i in stop:
    x = i.replace("\n",'')
    stopwords.append(x)

In [10]:
with open('ratings_train.txt',mode= 'r',encoding= 'UTF-8') as f:
    example = f.readlines()


In [11]:
def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

print(spacing_okt("대한민국은민주공화국이다.대한민국의모든권력은국민으로부터나온다."))

대한민국은 민주공화국이다. 대한민국의 모든 권력은 국민으로부터 나온다.


In [14]:
from bs4 import BeautifulSoup
sentences = []
labels = []
for items in example:
    sentence = items.replace(',', ' , ')
    sentence = sentence.replace('.', ' . ')
    sentence = sentence.replace('-', ' - ') 
    sentence = sentence.replace('/', ' / ')
    label = sentence.split('\t')[-1]
    labels.append(label) 
    sentences.append(sentence.split('\t')[1:-1])

In [19]:
new_labels = []
for i in labels:
    x = i.replace('\n', '')
    new_labels.append(x)
new_sentence = []
for i in sentences:
    if len(i) != 0:
        x = str(i).replace('[','')
        x = x.replace(']', '')
        new_sentence.append(x)
    else:
        new_sentence.append(' ')

In [39]:
space_sentences = []
for i in new_sentence:
    x = spacing_okt(i)
    space_sentences.append(x)    

In [49]:
new_labels = new_labels[1:]
space_sentences = space_sentences[1:]
labels = new_labels

#불용어 삭제

In [45]:
import string
table = str.maketrans('', '',string.punctuation)
sentences = []
for i in space_sentences:
    sentence = i.replace(',', ' , ')
    sentence = sentence.replace('.', ' . ')
    sentence = sentence.replace('-', ' - ') 
    sentence = sentence.replace('/', ' / ')
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    sentences.append(filtered_sentence)

In [44]:
last_sentence =[]
for i in sentences:
    try:
        x1= i.replace(' ','')
        x2 = spacing_okt(x1)
        last_sentence.append(x2)
    except:
        pass

150000

In [51]:
train_size = 120000
train_sentences = sentences[0:train_size]
test_sentences = sentences[train_size:]
train_labels = labels[0:train_size]
test_labels = labels[train_size:]

In [52]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 20000, oov_token="<OOV>")

In [53]:
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [63]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_sequences
import tensorflow  as tf
training_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen = 10,padding='post',truncating='pre')

In [56]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences)

In [64]:
training_padded.shape

(120000, 10)

In [72]:
class lstm_model(tf.keras.Model):
    def __init__(self):
        super(lstm_model,self).__init__()
        self.embedding_layer = tf.keras.Sequential([
            tf.keras.layers.Embedding(20000,10)                    
        ])
        self.lstm_layer = tf.keras.Sequential([
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10))
        ])
        self.out_layer = tf.keras.Sequential([
            tf.keras.layers.Dense(24),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(1,activation = 'sigmoid')
        ])
    def call(self, x):
        x = self.embedding_layer(x)
        x = self.lstm_layer(x)
        x = self.out_layer(x)
        return x


In [73]:
model = lstm_model()
model.build(input_shape = (1,10))

NotImplementedError: Cannot convert a symbolic Tensor (sequential_13/bidirectional_4/forward_lstm_4/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
model 